## Лабораторная №6. Выбор признаков
## Задание
* Реализуйте 3 метода выбора признаков: встроенный, обёртку и фильтрующий.
* Примените реализованные методы на наборе данных SMS или castle-or-lock.
* Выведите первые 30 признаков (слов), выбранные каждым методом.
* Сравните полученные списки с любыми тремя библиотечными методами, отличными от реализованных вами. В этом пункте не обязательно брать один метод обёртку, один встроенный и один фильтрующий.
* Определите, как меняется качество работы различных (не менее трёх) классификаторов до и после выбора признаков каждым из методов.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import nltk
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression

# Data

1. Считываем файл 'SMS.tsv' в переменную 'data' с использованием разделителя '\t' (табуляция).
2. Создаем переменную 'Y', которая содержит закодированные с помощью one-hot encoding значения столбца 'class'. Значения 'ham' закодированы как 1, а все остальные значения (обычно 'spam') закодированы как 0.
3. Создаем переменную 'X', которая содержит значения столбца 'text' после применения функции 'preprocesstext'. Функция 'preprocesstext' выполняет предобработку текста, такую как удаление лишних пробелов и символов и приводит весь текст к нижнему регистру.
4. Скачиваем стоп-слова для английского языка из библиотеки NLTK.
5. Инициализируем TF-IDF векторизатор 'tfid' с максимальным количеством признаков равным 2000 и заданными стоп-словами.
6. Преобразует переменную 'X' с помощью TF-IDF векторизатора 'tfid', сохраняя результаты в переменную 'X'.

Таким образом, этот код выполняет предобработку текста и векторизацию с использованием TF-IDF для дальнейшего использования в задаче классификации спама и не спама в SMS-сообщениях.

(TF-IDF (Term Frequency-Inverse Document Frequency) векторизатор используется для преобразования текстовых документов в числовые вектора, которые можно использовать для обучения моделей машинного обучения или для анализа текста. Он представляет каждый документ как вектор, где каждая компонента вектора соответствует какому-либо термину или слову. TF-IDF учитывает как важность термина внутри документа (частота термина в документе), так и его важность внутри коллекции документов (обратная частота термина в коллекции). Это позволяет отличить важные термины, которые встречаются часто в данном документе, от общих терминов, которые встречаются часто во всей коллекции.)

In [5]:
def preprocess_text(text):
    result = ''
    for char in text:
        if char.isalpha() or char.isspace():
            result += char
    result = ' '.join(result.split())
    return result.lower()

data = pd.read_csv('data/SMS.tsv', delimiter='\t')
X, Y = data.text.map(preprocess_text), pd.get_dummies(data['class']).ham

nltk.download('stopwords')
tfid = TfidfVectorizer(max_features=2000, stop_words=stopwords.words('english'))
X = tfid.fit_transform(X)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Embedded method

1. Обучаем модель RandomForestClassifier на входных данных X и Y.
2. Вычисляем важности признаков, полученные от обученной модели Random Forest, с помощью метода featureimportances.
3. Получаем список всех признаков с помощью метода getfeaturenamesout() объекта tfid.
4. Сортируем список признаков по убыванию их важности, используя важности признаков в качестве ключа для сортировки.
5. Фильтруем список признаков, оставляя только те признаки, чья важность превышает среднюю важность всех признаков.
6. Сохраняем результат в переменную selectedfeatures.

In [6]:
clf = RandomForestClassifier().fit(X, Y)
feature_importance = clf.feature_importances_
features = tfid.get_feature_names_out()

mean_importance = np.mean(feature_importance)
embedded_method = [feature for feature, importance in sorted(zip(features, feature_importance), key=lambda x: x[1], reverse=True) if importance > mean_importance]

print(len(embedded_method), len(features))

315 2000


# Wrapper method

In [7]:
# Используем функцию `train_test_split` из библиотеки `sklearn.model_selection`,
# чтобы разделить данные на обучающий набор (`X_train`, `Y_train`) и
# тестовый набор (`X_test`, `Y_test`). Тестовый набор составляет 20% от исходного набора данных.
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
X_tr, X_ts = X_train.toarray(), X_test.toarray()

# Определяем функцию `get_matrix`, которая принимает матрицу `m` и список столбцов `col`,
# и возвращает новую матрицу, содержащую только выбранные столбцы из исходной матрицы.
def get_matrix(m, col):
    return [[vec[j] for j in col] for vec in m]

best_accuracy, best_features = 0.0, []

# Создаем модель `GaussianNB` с помощью которой будем обучать и предсказывать наши данные.
model = GaussianNB()

# Будем итерироваться циклом пока не получим 30 признаков.
while len(best_features) < 30:
    current_features = best_features.copy()
    # Перебираем все признаки
    for k in range (len(features)):
        # Для каждого признака проверяем, если его индекс (`k`) не содержится в лучших признаках.
        if k in best_features:
            continue
        # Копируем в новый список текущий список лучших признаков, и добавляем индекс текущего признака.
        vector = current_features.copy() + [k]
        # Обучаем `GaussianNB` на основе полученной матрицы из вектора и X_tr, и `Y_train`.
        # model = GaussianNB()
        model.fit(get_matrix(X_tr, vector), Y_train)
        # Предсказываем на основе полученной матрицы из вектора и X_ts.
        Y_pred = model.predict(get_matrix(X_ts, vector))
        # Вычисляем точность прогнозов, сравнивая их с фактическими значениями из тестового набора.
        current_accuracy = accuracy_score(Y_test, Y_pred)
        # Берем признаки с лучшей точностью.
        if current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_features = vector
        # print(k, best_features, len(best_features), best_accuracy)

# берем признаки
wrapped_method = [features[k] for k in best_features]

0 [0] 1 0.15515695067264573
1 [1] 1 0.15605381165919283
2 [2] 1 0.15964125560538117
3 [2] 1 0.15964125560538117
4 [4] 1 0.8457399103139014
5 [4] 1 0.8457399103139014
6 [4] 1 0.8457399103139014
7 [4] 1 0.8457399103139014
8 [4] 1 0.8457399103139014
9 [9] 1 0.8484304932735426
10 [9] 1 0.8484304932735426
11 [9] 1 0.8484304932735426
12 [9] 1 0.8484304932735426
13 [9] 1 0.8484304932735426
14 [9] 1 0.8484304932735426
15 [9] 1 0.8484304932735426
16 [9] 1 0.8484304932735426
17 [9] 1 0.8484304932735426
18 [9] 1 0.8484304932735426
19 [9] 1 0.8484304932735426
20 [9] 1 0.8484304932735426
21 [21] 1 0.8493273542600897
22 [21] 1 0.8493273542600897
23 [21] 1 0.8493273542600897
24 [21] 1 0.8493273542600897
25 [21] 1 0.8493273542600897
26 [21] 1 0.8493273542600897
27 [21] 1 0.8493273542600897
28 [21] 1 0.8493273542600897
29 [21] 1 0.8493273542600897
30 [21] 1 0.8493273542600897
31 [21] 1 0.8493273542600897
32 [21] 1 0.8493273542600897
33 [21] 1 0.8493273542600897
34 [21] 1 0.8493273542600897
35 [21] 1 0.

/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:515: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:516: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:516: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:516: RuntimeWarning: invalid value encountered in subtract
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)


615 [294] 1 0.8681614349775785
616 [294] 1 0.8681614349775785
617 [294] 1 0.8681614349775785
618 [294] 1 0.8681614349775785
619 [294] 1 0.8681614349775785
620 [294] 1 0.8681614349775785
621 [294] 1 0.8681614349775785
622 [294] 1 0.8681614349775785
623 [294] 1 0.8681614349775785
624 [294] 1 0.8681614349775785
625 [294] 1 0.8681614349775785
626 [626] 1 0.8708520179372198
627 [626] 1 0.8708520179372198
628 [626] 1 0.8708520179372198
629 [626] 1 0.8708520179372198
630 [626] 1 0.8708520179372198
631 [626] 1 0.8708520179372198
632 [626] 1 0.8708520179372198
633 [626] 1 0.8708520179372198
634 [626] 1 0.8708520179372198
635 [626] 1 0.8708520179372198
636 [626] 1 0.8708520179372198
637 [626] 1 0.8708520179372198
638 [626] 1 0.8708520179372198
639 [626] 1 0.8708520179372198
640 [626] 1 0.8708520179372198
641 [626] 1 0.8708520179372198
642 [626] 1 0.8708520179372198
643 [626] 1 0.8708520179372198
644 [626] 1 0.8708520179372198
645 [626] 1 0.8708520179372198
646 [626] 1 0.8708520179372198
647 [626

/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:515: RuntimeWarning: divide by zero encountered in log
  n_ij = -0.5 * np.sum(np.log(2.0 * np.pi * self.var_[i, :]))
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:516: RuntimeWarning: divide by zero encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:516: RuntimeWarning: invalid value encountered in divide
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)
/usr/local/lib/python3.10/dist-packages/sklearn/naive_bayes.py:516: RuntimeWarning: invalid value encountered in subtract
  n_ij -= 0.5 * np.sum(((X - self.theta_[i, :]) ** 2) / (self.var_[i, :]), 1)


Выходные данные были обрезаны до нескольких последних строк (5000).
929 [1778, 294, 1078, 192, 1279, 1151, 1283, 21, 412, 1686, 1806, 129, 996, 271, 1480, 80, 1006, 373, 1481, 1592, 1664, 1887, 8, 56, 275, 55, 57, 116] 28 0.9721973094170404
930 [1778, 294, 1078, 192, 1279, 1151, 1283, 21, 412, 1686, 1806, 129, 996, 271, 1480, 80, 1006, 373, 1481, 1592, 1664, 1887, 8, 56, 275, 55, 57, 116] 28 0.9721973094170404
931 [1778, 294, 1078, 192, 1279, 1151, 1283, 21, 412, 1686, 1806, 129, 996, 271, 1480, 80, 1006, 373, 1481, 1592, 1664, 1887, 8, 56, 275, 55, 57, 116] 28 0.9721973094170404
932 [1778, 294, 1078, 192, 1279, 1151, 1283, 21, 412, 1686, 1806, 129, 996, 271, 1480, 80, 1006, 373, 1481, 1592, 1664, 1887, 8, 56, 275, 55, 57, 116] 28 0.9721973094170404
933 [1778, 294, 1078, 192, 1279, 1151, 1283, 21, 412, 1686, 1806, 129, 996, 271, 1480, 80, 1006, 373, 1481, 1592, 1664, 1887, 8, 56, 275, 55, 57, 116] 28 0.9721973094170404
934 [1778, 294, 1078, 192, 1279, 1151, 1283, 21, 412, 1686, 1806, 1

# Filter method

In [8]:
filtered, features_indexes = [], []

# матрица коэффициентов корреляции между всеми парами признаков в Х.
matrix = abs(np.corrcoef(X.toarray(), rowvar=False))

# проитерируемся по всем элементам матрицы которые превышают среднее значение всех элементов матрицы, для фильтрации только значимах корреляций
for p in np.argwhere(matrix > matrix.flatten().mean()):
    i, j = tuple(p)
    # Если i и j равны, то это значит, что признак коррелирует с самим собой и он пропускается.
    # Если i не содержится в features_fl и не содержится в filtered,
    # то он добавляется в features_fl. j добавляется в filtered.
    if i == j:
        continue
    if i not in features_indexes and i not in filtered:
        features_indexes += [i]
        filtered += [j]

# Список признаков, которые не были удалены на основе коэффициента корреляции с фильтрацией.
filtered_method = [features[i] for i in range(len(features)) if i not in filtered]

# Вводим первые 30 признаков из каждого метода.

In [9]:
print(embedded_method[:30])
print(wrapped_method)
print(filtered_method[:30])

['call', 'free', 'txt', 'claim', 'mobile', 'stop', 'text', 'prize', 'reply', 'urgent', 'win', 'nokia', 'service', 'customer', 'tone', 'contact', 'chat', 'ppm', 'box', 'ringtone', 'guaranteed', 'cash', 'awarded', 'pmin', 'new', 'tcs', 'pobox', 'per', 'apply', 'draw']
['txt', 'claim', 'mobile', 'box', 'pobox', 'nokia', 'points', 'admirer', 'delivery', 'texts', 'urgent', 'bak', 'ltgt', 'charge', 'service', 'apply', 'mail', 'currently', 'services', 'starts', 'tc', 'welcome', 'accidentally', 'always', 'chat', 'also', 'amazing', 'awaiting', 'award', 'cam']
['aiyah', 'appreciate', 'argue', 'argument', 'arms', 'arrested', 'asap', 'asks', 'asleep', 'assume', 'ate', 'attend', 'bag', 'bahamas', 'bak', 'balance', 'barely', 'basic', 'basically', 'bath', 'bathe', 'bathing', 'battery', 'bay', 'bcums', 'bday', 'becoz', 'bed', 'bedroom', 'beer']


# Сделаем все то же самое с библиотечными методами выбора признаков.

In [10]:
# Создаем объект модели SelectKBest с использованием метода chi2
# и задаем параметр, чтобы выбрать 300 наиболее значимых признаков.
# Обучаем модель на наборе данных и выводим первые 30 признаков.
model = SelectKBest(chi2, k=300)
model.fit(X, Y)
print(features[model.get_support()][:30])

# Преобразуем данные X с использованием выбранных признаков и сохраняем результат.
X_chi2 = model.transform(X)

['ac' 'account' 'action' 'admirer' 'age' 'alert' 'ampm' 'announcement'
 'anytime' 'apply' 'arcade' 'arrive' 'attempt' 'auction' 'await'
 'awaiting' 'award' 'awarded' 'bid' 'bluetooth' 'bonus' 'box'
 'btnationalrate' 'bx' 'call' 'caller' 'camcorder' 'camera' 'cash'
 'cashbalance']


In [11]:
# Создаем объект модели SelectFromModel с использованием модели LogisticRegression с параметрами penalty="l1", dual=False и solver='liblinear'.
# Обучает модель на наборе данных и выводим первые 30 признаков.
model = SelectFromModel(LogisticRegression(penalty="l1", dual=False, solver='liblinear').fit(X, Y), prefit=True)
print(features[model.get_support()][:30])

# Преобразует данные X с использованием выбранных признаков и сохраняет результат.
X_reg = model.transform(X)

['ac' 'access' 'admirer' 'age' 'apply' 'attempt' 'award' 'awarded' 'bid'
 'box' 'call' 'camera' 'cant' 'cash' 'charity' 'chat' 'choose' 'claim'
 'club' 'code' 'collect' 'collection' 'come' 'congrats' 'contact'
 'content' 'cost' 'credits' 'currently' 'customer']


In [12]:
# Создаем объект модели SelectKBest с использованием метода f_classif
# и задаем параметр, чтобы выбрать 300 наиболее значимых признаков.
# Обучаем модель на наборе данных и выводим первые 30 признаков.
model = SelectKBest(f_classif, k=300)
model.fit(X, Y)
print(features[model.get_support()][:30])

# Преобразует данные X с использованием выбранных признаков и сохраняет результат.
X_f_classif = model.transform(X)

['account' 'admirer' 'ae' 'age' 'ampm' 'announcement' 'ansr' 'apply'
 'attempt' 'auction' 'await' 'awaiting' 'award' 'awarded' 'bluetooth'
 'bonus' 'box' 'btnationalrate' 'bx' 'call' 'caller' 'camcorder' 'camera'
 'cash' 'cashbalance' 'cashin' 'cc' 'cd' 'cds' 'chance']


# Разбиваем данные на тренировочный и тестовый наборы

In [13]:
X_ = pd.DataFrame(X.toarray(), columns=features)
X_chi2_train, X_chi2_test, y_train, y_test = train_test_split(X_chi2, Y, test_size=0.2, random_state=42)
X_reg_train, X_reg_test, y_train, y_test = train_test_split(X_reg, Y, test_size=0.2, random_state=42)
X_f_train, X_f_test, y_train, y_test = train_test_split(X_f_classif, Y, test_size=0.2, random_state=42)
X_emb_train, X_emb_test, y_train, y_test = train_test_split(X_[embedded_method], Y, test_size=0.2, random_state=42)
X_wr_train, X_wr_test, y_train, y_test = train_test_split(X_[wrapped_method], Y, test_size=0.2, random_state=42)
X_fm_train, X_fm_test, y_train, y_test = train_test_split(X_[filtered_method], Y, test_size=0.2, random_state=42)

# Определим, как меняется качество работы различных (не менее трёх) классификаторов до и после выбора признаков каждым из методов.

In [14]:
# Logistic Regression

models = {
    'default': (X_train, X_test),
    'chi2': (X_chi2_train, X_chi2_test),
    'l1-based selection': (X_reg_train, X_reg_test),
    'f-test': (X_f_train, X_f_test),
    'random forest': (X_emb_train, X_emb_test),
    'Bayes wrapper': (X_wr_train, X_wr_test),
    'correlation': (X_fm_train, X_fm_test)
}

for model_name, (X_train, X_test) in models.items():
    accuracy = accuracy_score(y_test, LogisticRegression().fit(X_train, y_train).predict(X_test))
    print(f'{model_name}: {accuracy}')

default: 0.8663677130044843
chi2: 0.95695067264574
l1-based selection: 0.9533632286995516
f-test: 0.957847533632287
random forest: 0.9560538116591928
Bayes wrapper: 0.905829596412556
correlation: 0.9632286995515695


In [15]:
# KNN

models = {
    'default': (X_train, X_test),
    'chi2': (X_chi2_train, X_chi2_test),
    'l1-based selection': (X_reg_train, X_reg_test),
    'f-test': (X_f_train, X_f_test),
    'random forest': (X_emb_train, X_emb_test),
    'Bayes wrapper': (X_wr_train, X_wr_test),
    'correlation': (X_fm_train, X_fm_test)
}

for model_name, (X_train, X_test) in models.items():
    accuracy = accuracy_score(y_test, KNeighborsClassifier().fit(X_train, y_train).predict(X_test))
    print(f'{model_name}: {accuracy}')

default: 0.9246636771300448
chi2: 0.9381165919282511
l1-based selection: 0.9542600896860987
f-test: 0.9381165919282511
random forest: 0.9345291479820628
Bayes wrapper: 0.9623318385650225
correlation: 0.9246636771300448


In [16]:
# Decision tree

models = {
    'default': (X_train, X_test),
    'chi2': (X_chi2_train, X_chi2_test),
    'l1-based selection': (X_reg_train, X_reg_test),
    'f-test': (X_f_train, X_f_test),
    'random forest': (X_emb_train, X_emb_test),
    'Bayes wrapper': (X_wr_train, X_wr_test),
    'correlation': (X_fm_train, X_fm_test)
}

for model_name, (X_train, X_test) in models.items():
    accuracy = accuracy_score(y_test, DecisionTreeClassifier().fit(X_train, y_train).predict(X_test))
    print(f'{model_name}: {accuracy}')

default: 0.9623318385650225
chi2: 0.9623318385650225
l1-based selection: 0.9668161434977578
f-test: 0.9650224215246637
random forest: 0.9533632286995516
Bayes wrapper: 0.9596412556053812
correlation: 0.957847533632287
